In [0]:
import torch
from torchtext import data
from torchtext import datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [7]:
generate_bigrams(['AI', 'is', 'future', ])


['AI', 'is', 'future', 'is future', 'AI is']

In [0]:

torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', preprocessing = generate_bigrams)
LABEL = data.LabelField(dtype = torch.float)

In [0]:
train, test = datasets.IMDB.splits(TEXT, LABEL)

train, valid = train.split()


In [11]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train)

.vector_cache/glove.6B.zip: 862MB [06:27, 2.23MB/s]                           
100%|█████████▉| 398465/400000 [00:15<00:00, 26635.75it/s]

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64, 
    device = device)

In [0]:
class FastText(nn.Module):
    def __init__(self, v_size, emb_dim, out_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(v_size, emb_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(emb_dim, out_dim)
        
    def forward(self, text):
        
        embedded = self.embedding(text)
        
        embedded = embedded.permute(1, 0, 2)
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
                
        return self.fc(pooled)

In [0]:
in_dim = len(TEXT.vocab)
emb_dim = 100
out_dim = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(in_dim, emb_dim, out_dim, TEXT.vocab.stoi[TEXT.pad_token])

In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,301 trainable parameters


In [22]:

pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.8669,  0.1136, -1.4781,  ..., -1.4515, -1.9527,  0.7645],
        [-1.4479, -0.5089, -0.7270,  ..., -0.8029, -0.2880,  0.2453],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.8969, -0.2661, -0.0853,  ..., -0.2921, -0.1018,  0.5744],
        [-0.9298, -0.5587, -1.0932,  ..., -0.0512,  0.8476,  0.9561],
        [ 1.1092, -0.1739, -0.1906,  ...,  0.0627,  0.8374, -0.0685]])

In [0]:
model.embedding.weight.data[TEXT.vocab.stoi[TEXT.unk_token]] = torch.zeros(emb_dim)
model.embedding.weight.data[PAD_IDX] = torch.zeros(emb_dim)

In [0]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(predictions, y):
    rounded = torch.round(torch.sigmoid(predictions))
    correct = (rounded == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def Train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [31]:
best_valid_loss = float('inf')

for epoch in range(5):

    start_time = time.time()
    
    train_loss, train_acc = Train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'ThirdModel.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 8s
	Train Loss: 0.380 | Train Acc: 88.18%
	 Val. Loss: 0.382 |  Val. Acc: 86.69%
Epoch: 02 | Epoch Time: 0m 8s
	Train Loss: 0.342 | Train Acc: 89.31%
	 Val. Loss: 0.392 |  Val. Acc: 87.18%
Epoch: 03 | Epoch Time: 0m 8s
	Train Loss: 0.309 | Train Acc: 90.56%
	 Val. Loss: 0.402 |  Val. Acc: 87.70%
Epoch: 04 | Epoch Time: 0m 8s
	Train Loss: 0.284 | Train Acc: 91.35%
	 Val. Loss: 0.416 |  Val. Acc: 88.31%
Epoch: 05 | Epoch Time: 0m 8s
	Train Loss: 0.261 | Train Acc: 92.04%
	 Val. Loss: 0.434 |  Val. Acc: 88.47%


In [0]:
 import spacy
nlp = spacy.load('en')

def sentiment_prediction(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [34]:
sentiment_prediction(model, "AI is terrible")


0.0

In [36]:
sentiment_prediction(model, "AI is future")


1.0